## Set Covering Path Search

In this notebook, I referenced the professor's notebook, modifying his implementations and adding my own


In [1]:
from queue import PriorityQueue, SimpleQueue, LifoQueue
from random import random
from functools import reduce
from collections import namedtuple
from pprint import pprint
import numpy as np

Sets the initial constant for the problem

In [2]:
PROBLEM_SIZE = 5
NUM_SETS = 10

Create randomly the SETS array of tiles with length PROBLEM_SIZE, and with a probability of 30% to have a true value fo each element. <br>
And I choose how **state** this rappresentation **({sets_taken},{sets_not_taked})** with **sets_taken** and **sets_not_taked** the index of the sets.

In [3]:
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple("State", ["taken", "not_taken"])

To evaluate the goal:
1. take all the sets selected in the current state
2. apply a or between all this sets with the reduce function
3. return true just if we have all true in the resulted array

Note: we can have overlapping, it is not a limitation in the problem, so the or is a good choice

In [4]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

Now I check if the problem is solvable, because if I cannot solve it with all the sets the problem is not solvable

In [5]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

## Uninformed Search
1. Depth First Search
2. Breadth First Search
3. Uniform Cost Search (Djkastra)

Define a function search that with a frontier apply the path search and return the goal state

In [6]:
def search(frontier):
    current_state = frontier.get()

    # to count the number of sets put in the frontier
    counter = 0
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put(new_state)
        current_state = frontier.get()

    print(f"Solved in {counter} steps")
    return current_state

With **SimpleQueue** we are solving the problem with a **Breadth First**, in this way we are solving the problem with the optimal solution in the sense of **minimum number of tiles to covering all the line**.
With a **LifoQueue** I'm searching the solution with the **Depth First** that don't find the optimal solution but it is quicker.

In [7]:
# define a SimpleQueue, and LifoQueue how frontiers
fifo = SimpleQueue()
lifo = LifoQueue()
# put the initial state (taken is empty and not taked is all the sets) in the frontiers
fifo.put(State(set(), set(range(NUM_SETS))))
lifo.put(State(set(), set(range(NUM_SETS))))

breadth_solution_state = search(fifo)
depth_solution_state = search(lifo)

Solved in 111 steps
Solved in 8 steps


In [8]:
print(
    f"Solution with a Breadth Search:\n{breadth_solution_state}\nSolution with a Depth Search:\n{depth_solution_state}"
)

Solution with a Breadth Search:
State(taken={0, 2, 4}, not_taken={1, 3, 5, 6, 7, 8, 9})
Solution with a Depth Search:
State(taken={2, 3, 4, 5, 6, 7, 8, 9}, not_taken={0, 1})


In this case we don't have any type of cost (we are assuming the number of sets), if we define a different cost, for example a solution with the smallest number of overlap. In the sense that the **cost is the total number of true in taken sets**. To solve the problem , in this case we have to add the cost how key and use a PriorityQueue to order the queue to solve a **Dijkstra**. In this way we are searching for a solution with the **minimum number of overlapping**.

In [9]:
def cost_function(state):
    return sum([np.sum(SETS[i]) for i in state.taken])

In [10]:
def search_dijkstra(frontier):
    current_state = frontier.get()

    # to count the number of sets put in the frontier
    counter = 0
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((cost_function(new_state), new_state))
        current_state = frontier.get()[1]

    print(f"Solved in {counter} steps")
    return current_state

In [11]:
# define a PriorityQueue how frontiers
frontier = PriorityQueue()

# put the initial state (taken is empty and not taked is all the sets) in the frontiers
frontier.put(State(set(), set(range(NUM_SETS))))

dijkstra_solution_state = search_dijkstra(frontier)

Solved in 2548 steps


In [12]:
dijkstra_solution_state

State(taken={1, 2, 4, 5}, not_taken={0, 3, 6, 7, 8, 9})

In [13]:
pprint([SETS[i] for i in dijkstra_solution_state.taken])

[array([False, False, False,  True, False]),
 array([False,  True,  True, False, False]),
 array([ True, False,  True, False,  True]),
 array([False, False, False, False, False])]


## Informed Search
- Greedy Best First Search
- A* Search

Now I want to implemente a version of **Informed search**, so with a **Euristich** that estimate the distances from a state to the goal, and use it to modify the search.<br>
I define **h(n)** heuristic of node n with the **number of false that I have considering the OR of all the taken sets**, i.e. number of true that are missing for the goal.

**Greedy Best First** using just the euristich **h(n)**. <br>


In [14]:
def h(state):
    return PROBLEM_SIZE - sum(
        reduce(np.logical_or, [SETS[i] for i in state.taken])
    )

In [15]:
def greedy_best_search(frontier):
    current_state = frontier.get()

    steps = 0
    while not goal_check(current_state):
        steps += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((h(new_state), new_state))
        _, current_state = frontier.get()
    print(f"Solved in {steps} steps and with {len(current_state.taken)} tiles")
    return current_state

In [16]:
# define a PriorityQueue how frontiers
frontier = PriorityQueue()

# put the initial state (taken is empty and not taked is all the sets) in the frontiers
frontier.put(State(set(), set(range(NUM_SETS))))

greedy_solution = greedy_best_search(frontier)
pprint([SETS[i] for i in greedy_solution.taken])

Solved in 3 steps and with 3 tiles
[array([False, False,  True,  True, False]),
 array([False,  True,  True, False, False]),
 array([ True, False,  True, False,  True])]


## Lab1 with A* - 19/10/2023
**A\* Search** where the evalutation function **f(n) = g(n) + h(n)**:<br>
1. **g(n)** the cost from start to node n, so the **number of tiles in the taken set**.<br>

For h(n) I implemented two different h(n):<br>
1. **h(n)** the same heuristic function used in the greedy best first.<br>
2. **adm_h(h)** admissible heuristics to guarantee the optimal solution.



The **h(n)** heuristic is not admissible because it considers the number of missing values as cost; this means considering the **pessimistic** case, in which to cover the whole line each set would have to have a single true. But for an admissible heuristic we need an **optimistic** heuristic. So I defined **adm_h(n)**, this heuristic considers the minimum number of tiles, belonging to the untaken sets, that I need to cover the missing values.

Note: for the adm_h(n) I modified the prof's, I considered only the sets not taken, because considering all the sets, even the ones taken, is useless because we want to consider only the missing values that obviously the ones taken cannot cover.

In [17]:
def adm_h(state):
    already_covered = covered(state)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    if missing_size == 0:
        return 0
    candidates = sorted(
        (
            sum(np.logical_and(SETS[i], np.logical_not(already_covered)))
            for i in state.not_taken
        ),
        reverse=True,
    )
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

In [18]:
def evaluation_function(state):
    return len(state.taken) + h(state)

In [19]:
def evaluation_function_admissible(state):
    return len(state.taken) + adm_h(state)

In [20]:
def A_star(frontier, evaluation_function):
    current_state = frontier.get()

    # to count the number of sets put in the frontier
    counter = 0
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put(
                (
                    evaluation_function(new_state),
                    new_state,
                )
            )
        current_state = frontier.get()[1]

    print(f"Solved in {counter} steps")
    return current_state

In [21]:
# define a PriorityQueue how frontiers
frontier = PriorityQueue()
frontier2 = PriorityQueue()

# put the initial state (taken is empty and not taked is all the sets) in the frontiers
frontier.put(State(set(), set(range(NUM_SETS))))
frontier2.put(State(set(), set(range(NUM_SETS))))

A_star_solution = A_star(frontier, evaluation_function)
A_star_admissible_solution = A_star(frontier2, evaluation_function_admissible)

Solved in 5 steps
Solved in 8 steps


In [22]:
print("A* with not admissible heuristic:")
pprint([SETS[i] for i in A_star_solution.taken])

A* with not admissible heuristic:
[array([False, False,  True,  True, False]),
 array([False,  True,  True, False, False]),
 array([ True, False,  True, False,  True])]


In [23]:
print("A* with admissible heuristic:")
pprint([SETS[i] for i in A_star_admissible_solution.taken])

A* with admissible heuristic:
[array([False,  True,  True, False, False]),
 array([ True, False,  True, False,  True]),
 array([ True, False, False,  True, False])]


### Results of A* compared to Breadth First's for this run:
1. **Breadth First Search:** optimal solution **3 tiles** in **111 steps**
2. **A\* search** with not admissible heuristic: **3 tiles** in **5 steps**
3. **A\* search** with admissible heuristic: **3 tiles** in **8 steps**

As you can see, although with the not admissible heuristics it is not guaranteed to have the optimal solution, for this simple problem we very often get the optimal solution and with a very low number of steps.<br>Both implementations of A* lead to a good reduction in steps compared to Breadth First, which controls all possible combinations for each layer.
